In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, ConcatDataset
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, transforms
from PIL import Image
import os
import csv
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

from models import *
from function import *

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cpu


In [5]:
def Plot(title, ylabel, epochs, train_loss, valid_loss):
    plt.figure()
    plt.title(title)
    plt.xlabel('epochs')
    plt.ylabel(ylabel)
    plt.plot(epochs, train_loss)
    plt.plot(epochs, valid_loss)
    plt.legend(['train', 'valid'], loc='upper left')
    plt.savefig('PIC' + title + ".png")

In [3]:
train_tfm = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [2]:
# Argument
batch_size = 32
epochs = 5

In [4]:
train_set = trainDataset('retina-train', 'train_labels.csv', train_tfm)
train_set_size = int(len(train_set) * 0.8)
train_set, valid_set = torch.utils.data.random_split(train_set, [train_set_size, len(train_set) - train_set_size])

test_set = testDataset('retina-test', train_tfm)
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=False, num_workers=0)
valid_loader = DataLoader(valid_set, batch_size=batch_size, shuffle=False, num_workers=0)
test_loader = DataLoader(test_set, batch_size=batch_size, shuffle=False, num_workers=0)

In [6]:
model = VGG16(3)
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.1)
scheduler = lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)
softmax = nn.Softmax(dim = -1)


train_loss = []
train_accs = []
valid_loss = []
valid_accs = []
    
for epoch in range(epochs):
    tl = 0
    ta = 0
    total = 0
    model.train()
    for batch in tqdm(train_loader):
        imgs, labels = batch
        imgs, labels = imgs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(imgs)
     
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        tl += loss.item()
        _, preds = torch.max(outputs, 1)
        total += labels.size(0)
        ta += (preds == labels).sum().item()
    scheduler.step()
    print(f'Epoch {epoch+1} / {epochs} | train_accs = {ta / total} & train_loss = {tl / total}')

    model.eval()
    vl = 0.0
    va = 0
    total = 0
    with torch.no_grad():
        for batch in tqdm(valid_loader):
            imgs, labels = batch
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            vl += loss.item()
            _, preds= torch.max(outputs, 1)
            total += labels.size(0)
            va += (preds == labels).sum().item()

    print(f'Epoch {epoch+1} / {epochs} | validation_accs = {va / total} & validation_loss = {vl / total}')
    train_loss.append(tl / total)
    train_accs.append(ta / total)
    valid_loss.append(vl / total)
    valid_accs.append(va / total)

torch.save(model.state_dict(), './checkpoints/proj1_baseline.pth')
Plot('Loss_Curve', 'Loss', range(1, epochs+1), train_loss, valid_loss)
Plot('Accuracy_Curve', 'Accuracy', range(1, epochs+1), train_accs, valid_accs)

  0%|          | 0/53 [00:14<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
preds_list = []

with torch.no_grad():
    for batch in tqdm(test_loader):
        imgs = batch
        imgs = imgs.to(device)
        outputs = model(imgs)
        _, preds= torch.max(outputs, 1)
        preds_list += preds.tolist()

name = sorted(os.listdir("retina-test"))
for i in range(len(name)):
    name[i], _ = name[i].split('.')

submission ={"image" : name, "level" : preds_list}
file = pd.DataFrame(submission)
file.to_csv("submission1.csv", index = False)

100%|██████████| 17/17 [00:01<00:00, 10.58it/s]
